In [1]:
import pylab as plt
import numpy as np
import swot_ssh_utils as swot
from cartopy import crs as ccrs
from netCDF4 import Dataset
import xarray as xr
import sys as sys
import glob as glob
import datetime

plt.rcParams.update({'font.size': 18,'savefig.facecolor':'white'})

In [ ]:
# Exemple of file read with E(f) data 

numf=6  # number of frequencies summed over 

# NB this file is also at: 
# https://data-ww3.ifremer.fr/PROJECT/SWOT/GLOBAL/work2023-01-01T00_2023-07-01T00/FIELD_NC/GLOB-30M/SWOT_WW3-GLOB-30M_202306_ef.nc
pth_swot='/media/ardhuin/FabLinux/SWOT2/'  # this is for the updated data
mypath1='/media/ardhuin/FabLinux/WW3-SWOT/GLOBAL/';nskip=3
mypath='/media/ardhuin/FabLinux/WW3-SWOT/GLOBAL_REF102040/';nskip=1
#mypath2='/media/ardhuin/FabLinux/WW3-CCI/'

addSWOTtracks=0

file_list = sorted(glob.glob(mypath+'*202308_ef.nc'));
nfiles=len(file_list)

print('We have ',nfiles,' files.')

indf=0
conval=np.arange(0,9,1) #[0.05, 0.1, 0.2, 0.4, 0.6, 1.0, 1.5, 2., 3., 4.] #, 6., 8., 10]

#fig=plt.figure(figsize=(16,8))
fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(16,8))
#spec = mpl.gridspec.GridSpec(ncols=2, nrows=1,width_ratios=[6, 5])
plt.subplots_adjust(left=0.05,bottom=0.1, top=0.92,wspace=0.1,right=0.99)
import warnings
warnings.filterwarnings("ignore")

# defines contour levels for map
conval=[0.05, 0.1, 0.2, 0.4, 0.6, 1.0, 1.5, 2., 3., 4., 6., 8., 10]


for file in file_list:
    print('File:',file)
    ds_ww3 = xr.open_dataset(file)
# Selects time step 45 and frequency 3 and convert to m^2/Hz 
    f=ds_ww3.f
    print('We are going tu sum over these frequencies:',f[0:numf].values,1/f[0:numf].values)
    indfs=np.arange(0,numf,1);
    df=f*(1.1-1/1.1)*0.5  # warning: this assumes XFR=1.1 ... may not be true for all simulations!!
    nf=np.size(f)
    weight_freq=np.zeros(nf)+1
    [nt,nf,ny,nx]=np.shape(ds_ww3.ef)
    for it0 in range(nt//nskip):
       it=it0
       its=it*nskip
       plt.clf()
       Etot=np.zeros((ny,nx))
       for ind in indfs:
         E2D=ds_ww3.ef[its,ind,:,:]
         Ef = 10**(E2D)-1E-12
         Etot = Etot + Ef*df[ind]*weight_freq[ind]
       Hs = 4*np.sqrt(Etot)
    
       
       ax = plt.axes(projection=ccrs.PlateCarree())     
       cm=plt.pcolormesh(ds_ww3.longitude, ds_ww3.latitude  , np.log10(Hs), shading ='auto',vmin = np.log10(0.05), vmax =np.log10(10),rasterized=True)

       
       cbar=plt.colorbar(shrink = 0.8, label =r"$H_{18}$ (m)")

       cbar.set_ticks(np.log10(conval))
       cbar.set_ticklabels(conval)
       gl = ax.gridlines(linestyle="--")
       gl.bottom_labels = True
       gl.top_labels= False
       gl.left_labels= True
       gl.right_labels=False
       gl.xlines = False
       
       tshift=0. #8*86400   # due to WW3 bug related to non-gregorian calendar ... 
       tnow=ds_ww3['time'].values[its]
       time_change = tshift #(iCycle-5)*cycle_length
       new_time = tnow+np.timedelta64(int(np.round(time_change*1e9)), 'ns')
       times=str(new_time)[0:4]+str(new_time)[5:7]+str(new_time)[8:10]
       times2=str(new_time)[0:13]
       #fig.suptitle(times2)
       if (np.mod(it,8)==0):
          print(its,ds_ww3.time[its].values)
          lonswot=[]
          latswot=[]
          lontag=np.zeros(100)
          lattag=np.zeros(100)
          if (addSWOTtracks==1): 
              taglist=[]
          
              file_tracks = sorted(glob.glob(pth_swot+'SWOT_L2_LR_SSH_WindWave_???_???_'+times+'*.nc'))
              indtrack=0
              group0=''
              for file_track in file_tracks:
                  tags=file_track.split(sep='_')
                  Cycle=int(tags[-6])
                  Pass=int(tags[-5])
            
              group='%03d-%03d'%(Cycle, Pass)
              if indtrack==0:
                group0=group
              ds = xr.open_dataset(file_track)
              alllat=ds.latitude[:,34].values
              lon360=ds.longitude[:,34].values
              lon180_0=np.where(lon360>180.,lon360-360.,lon360)
              ntswot=np.size(ds.time.values)
              tshift=0 
       
              newlons=lon180_0[0:ntswot:20]
              newlats=alllat[0:ntswot:20]
              lonswot.extend(newlons)
              latswot.extend(newlats)
              indl=np.where(newlats*newlats[0] < 0)[0] 
              lontag[indtrack]=newlons[indl[0]]
              lattag[indtrack]=5*np.sign(newlats[0])
              taglist.append(Pass)
              indtrack=indtrack+1
       H18max=f'{np.max(Hs):05.2f}'
       ax.set_title(times2+' tracks of day:'+group0+' to '+group+', max(H18)='+H18max+'m')
       if (addSWOTtracks==1): 
           ax.scatter(lonswot,latswot, c='r',marker='.',s=2) 
           for ind in range(indtrack):
               ax.text(lontag[ind],lattag[ind],taglist[ind],fontsize=15,color='r')
       pngf='temp'+f'{it:03d}'+'.png'
       fig.savefig(pngf,dpi=50) 
       img = plt.imread(pngf).astype(np.uint8) #.astype(np.uint8)


We have  1  files.
File: /media/ardhuin/FabLinux/WW3-SWOT/GLOBAL_REF102040/SWOT_WW3-GLOB-30M_202308_ef.nc
We are going tu sum over these frequencies: [0.0339     0.03729    0.041019   0.0451209  0.04963299 0.05459629] [29.498526 26.816841 24.378948 22.162678 20.14789  18.316263]
0 2023-08-01T00:00:00.000000000
8 2023-08-02T00:00:00.000000000
16 2023-08-03T00:00:00.000000000
24 2023-08-04T00:00:00.000000000
32 2023-08-05T00:00:00.000000000
40 2023-08-06T00:00:00.000000000
48 2023-08-07T00:00:00.000000000
56 2023-08-08T00:00:00.000000000
64 2023-08-09T00:00:00.000000000
72 2023-08-10T00:00:00.000000000
80 2023-08-11T00:00:00.000000000
88 2023-08-12T00:00:00.000000000
96 2023-08-13T00:00:00.000000000
104 2023-08-14T00:00:00.000000000
112 2023-08-15T00:00:00.000000000
120 2023-08-16T00:00:00.000000000
128 2023-08-17T00:00:00.000000000
136 2023-08-18T00:00:00.000000000
144 2023-08-19T00:00:00.000000000
152 2023-08-20T00:00:00.000000000
160 2023-08-21T00:00:00.000000000
168 2023-08-22T00:00: